In [ ]:
rc('figure',figsize=(16,4))
rc('font',size=12)
rc('text',usetex=False)
rc('image', cmap='viridis')
import numpy as np

In [ ]:
#### This is the analytical synthesized for a pixel at the focal plane center 
#### without accounting for the primry beam (just the mukltple peaks)
def sb_noprim(th_deg, nu):
    th = np.radians(th_deg)
    lam = 3e8/nu
    P = 20
    deltax = 0.013
    df = 300.
    abscissa = np.pi * deltax/lam * th
    sb = np.sin(P*abscissa)**2 / np.sin(abscissa)**2
    return sb/np.max(sb)

In [ ]:
## With Multiband = False  => using a QubicInstrument
import qubic
d = qubic.qubicdict.qubicDict()
d.read_from_file('/Users/hamilton/Python/GitQubicMousset/qubic/qubic/dicts/global_source_oneDet.dict')
d['multiband'] = False
d['beam_shape'] = 'multi_freq'   # fitted_beam, gaussian, multi_freq
print('Beam Shape:',d['beam_shape'])


freqs = np.array([1.32638868e+11, 1.50398340e+11, 1.67001321e+11])
nn = 1000
th= np.linspace(-30,30, nn)
ph = 0

allbeams = np.zeros((len(freqs), nn))
for i in range(len(freqs)):
    d['filter_nu'] = freqs[i]
    q = qubic.QubicInstrument(d)
    allbeams[i,:] = q.primary_beam(np.radians(np.abs(th)),ph)
        
    subplot(1,2,1)
    p=plot(th, allbeams[i,:],'--', label='nu={0:6.1f}'.format(freqs[i]/1e9))
    plot(th, allbeams[i,:]*sb_noprim(th, freqs[i]), color=p[0].get_color())
#     subplot(1,2,2)
#     p=plot(th, allbeams[i,:],'--', label='nu={0:6.1f}'.format(freqs[i]/1e9))
#     plot(th, allbeams[i,:]*sb_noprim(th, freqs[i]), color=p[0].get_color())
#     yscale('log')
#     ylim(1e-3,0)
    title('d[beam_shape]='+d['beam_shape']+' , Multiband='+str(d['multiband']))
legend()


In [ ]:
q.primary_beam??

In [ ]:
# With multiband = True code have to be written differently
import qubic
d = qubic.qubicdict.qubicDict()
d.read_from_file('/Users/hamilton/Python/GitQubicMousset/qubic/qubic/dicts/global_source_oneDet.dict')
d['multiband'] = True
d['beam_shape'] = 'multi_freq'   # fitted_beam, gaussian, multi_freq
print('Beam Shape:',d['beam_shape'])

q = qubic.QubicMultibandInstrument(d)
nsub = len(q)
nus = np.zeros(nsub)
for i in range(nsub): nus[i] = q[i].filter.nu

indices = [0, 6, 11]

for i in range(len(indices)):
    allbeams[i,:] = q[indices[i]].primary_beam(np.radians(np.abs(th)),ph)
    subplot(1,2,1)
    p=plot(th, allbeams[i,:], '--', label='nu={0:6.1f}'.format(nus[i]/1e9))
    plot(th, allbeams[i,:]*sb_noprim(th, freqs[i]), color=p[0].get_color())
    subplot(1,2,2)
    p=plot(th, allbeams[i,:], '--', label='nu={0:6.1f}'.format(nus[i]/1e9))
    plot(th, allbeams[i,:]*sb_noprim(th, freqs[i]), color=p[0].get_color())
    yscale('log')
    ylim(1e-3, 0)
    title('d[beam_shape]='+d['beam_shape']+' , Multiband='+str(d['multiband']))
legend()
